In [1]:
!pip install -r ./custom_vdb/requirements.txt -q


[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [0]:
# !cd custom_vdb && python create_vectordb.py

In [1]:
import sys 
import json 
import datarobot as dr
client = dr.Client()

## test out the model
sys.path.append("./custom_vdb")
from custom_vdb import custom
model = custom.load_model("./custom_vdb")
relevant_docs = custom.score_unstructured(model, json.dumps(dict(question = "boosted regression tasks")), query = {})
json.loads(relevant_docs[0])["relevant"][0]

/home/notebooks/storage/custom_vdb/custom.py:18: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(
/home/notebooks/storage/.venv/lib64/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/notebooks/storage/.venv/lib64/python3.11/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.


'Frequency-Severity eXtreme Gradient Boosted Trees'

In [3]:
from datarobot.models.runtime_parameters import RuntimeParameter, RuntimeParameterValue
## create an entry in the custom model workshop for our vdb
cm = dr.CustomInferenceModel.create(
    name = "byo vdb",
    target_type = dr.enums.TARGET_TYPE.UNSTRUCTURED, 
    language = "python",
    network_egress_policy = dr.enums.NETWORK_EGRESS_POLICY.PUBLIC,
                                    )

In [4]:
## grab the prediction environment
environment = [env for env in dr.ExecutionEnvironment.list() if env.name == "[DataRobot] Python 3.11 GenAI"].pop()
## add a version to the entry in the custom model workshop. 
cmv = dr.CustomModelVersion.create_clean(cm.id, base_environment_id=environment.id,
                                         folder_path = "/home/notebooks/storage/custom_vdb", 
                                         network_egress_policy = dr.enums.NETWORK_EGRESS_POLICY.PUBLIC)

In [5]:
## build the custom model dependecies this will take some time. 
try:
        build = dr.CustomModelVersionDependencyBuild.start_build(cm.id, cmv.id)
except Exception as e:
    print(e)

In [11]:
## register the custom model version in the dr model registry 
registered_model_version = dr.RegisteredModelVersion.create_for_custom_model_version(
    custom_model_version_id =  cmv.id, 
    name = "byo vdb 20250107", 
    description = "externally created vdb", 
    registered_model_name = "datarobot modeling task external vdb ", 
    registered_model_description = "datarobot modeling task external vdb "
)

For some reason drx.deploy does NOT support deployment with Serverless Prediction environments.  Use the following to make a raw request.

In [12]:
prediction_environment = dr.PredictionEnvironment.list().pop()
prediction_environment

PredictionEnvironment('67643686a012eb0b2a8e533f', 'Serverless Compute', 'datarobotServerless', 'Datarobot Serverless Compute supports both GenAI and Predictive deployments')

In [14]:
client = dr.Client() 
payload = {
    "modelPackageId":registered_model_version.id,
    "predictionEnvironmentId": prediction_environment.id,
    "label":"datarobot modeling task vdb",
    "importance":"LOW"}
dep_req = client.post("deployments/fromModelPackage", data = payload)

In [17]:
## wait for the deployment to create
response = client.get(dep_req.headers["location"].replace("https://app.datarobot.com/api/v2/","")).json()
while response.get("status") == "INITIALIZED":
    response = client.get(dep_req.headers["location"].replace("https://app.datarobot.com/api/v2/","")).json()
response = client.get(dep_req.headers["location"].replace("https://app.datarobot.com/api/v2/","")).json()
deployment_id = response["id"]
print(response)

{'id': '677d9f78787fc622cab4abfd', 'label': 'datarobot modeling task vdb', 'description': None, 'createdAt': '2025-01-07T21:41:13.463000Z', 'model': {'id': '677d9b0a2592739b28e40e43', 'type': 'byo vdb', 'targetName': None, 'targetType': 'Unstructured', 'projectId': None, 'projectName': None, 'unsupervisedMode': False, 'unstructuredModelKind': True, 'buildEnvironmentType': 'Python', 'deployedAt': '2025-01-07T21:41:13.464000Z', 'customModelImage': {'customModelId': '677d9b06ac6c0024cb8d3067', 'customModelName': 'byo vdb', 'customModelVersionId': '677d9b0a2592739b28e40e43', 'customModelVersionLabel': 'v1.0', 'executionEnvironmentId': '64d2ba178dd3f0b1fa2162f0', 'executionEnvironmentName': '[DataRobot] Python 3.11 GenAI', 'executionEnvironmentVersionId': '6717f8e0f7c696f62730c286', 'executionEnvironmentVersionLabel': 'v12'}, 'isDeprecated': False, 'prompt': None}, 'status': 'active', 'capabilities': {'supportsModelReplacement': True, 'supportsTargetDriftTracking': False, 'supportsFeatureDr

In [18]:
## test the deployment
import sys
import json
import requests
import pprint
import os
 
API_URL = 'https://app.datarobot.com/api/v2/deployments/{deployment_id}/predictionsUnstructured'    # noqa
API_KEY = os.environ["DATAROBOT_API_TOKEN"]
DEPLOYMENT_ID = deployment_id 

# Set HTTP headers. The charset should match the contents of the file.
headers = {
    'Content-Type': '{};charset={}'.format("application/json", "UTF-8"),
    'Authorization': 'Bearer {}'.format(API_KEY),
}

url = API_URL.format(deployment_id=DEPLOYMENT_ID)

# Make API request for predictions
predictions_response = requests.post(
    url,
    data= json.dumps(dict( question = "datarobot task for gradient boosting")),
    headers=headers,
)
    # Return raw response content
pprint.pprint( predictions_response.json() )



{'question': 'datarobot task for gradient boosting',
 'references': [{'bp_workshop_description': 'Gradient Boosting Classifier '
                                            '(xgboost) with Early-Stopping and '
                                            'Unsupervised Learning features. '
                                            'Gradient Boosting Machines are a '
                                            'cutting-edge algorithm for '
                                            'fitting extremely accurate '
                                            'predictive models. GBMs have won '
                                            'a number of recent predictive '
                                            'modeling competitions.',
                 'bp_workshop_name': 'eXtreme Gradient Boosted Trees '
                                     'Classifier with Early Stopping and '
                                     'Unsupervised Learning Features',
                 'category': 'Binary Cl

## add the vectordb so it can be used in the datarobot playground

In [2]:
external_vdb_validation = dr.models.genai.CustomModelVectorDatabaseValidation.create(
    prompt_column_name="question", 
    target_column_name="relevant",
    deployment_id="677d9f78787fc622cab4abfd", 
    wait_for_completion=True
)

In [3]:
external_vdb_validation.validation_status

'PASSED'

In [5]:
from datarobot.models.genai.vector_database import VectorDatabase

vdb = VectorDatabase.create_from_custom_model(
    name="DR External Vector Database",
    use_case="6769a8674d37762aa0576ccf",
    validation_id=external_vdb_validation.id
)

In [6]:
assert vdb.execution_status == "COMPLETED"

In [7]:
vdb.id

'677e638c0235157b6f9f0223'